## <center>Processamento de Linguagem Natural em Textos de Mídias Sociais: Fundamentos, Ferramentas e Aplicações</center>

### <center>XXVIII Simpósio Brasileiro de Sistemas Multimídia e Web (WebMedia 2022)</center>

<br></br>

<center>Frances A. Santos (UNICAMP), Jordan Kobellarz (UTFPR), Fábio R. de Souza (USP), Leandro A. Villas (UNICAMP), Thiago H. Silva (UTFPR)</center>

<br></br>

<center>Curitiba, PR</center>
<center>07 de Novembro de 2022</center>

<a href="https://colab.research.google.com/github/webmedia2022-nlp/course-code/blob/main/NLP_WebMedia2022.ipynb" target="_parent"><img style="float: right;" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir no Colab"/></a>

## Agenda

- Introdução 
- Texto de mídias sociais: suas principais características e como coletá-los
- Pré-processamento textual
- Representação de textos utilizando vetores numéricos
- Modelagem e extração de conhecimento
- Compreensão semântica e emocional
- Possíveis aplicações
- Perguntas & Respostas

## Introdução

<img src="figs/social-media.jpeg" style="float: center; zoom:50%;" />

- Mídias sociais ..
- To Do


In [1]:
pip install --no-cache-dir -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import getpass
import warnings
import pathlib
import os 
from tqdm import tqdm

from ExtracaoDados import ExtracaoDados
from PreProcessamento import PreProcessamento
from ModelosRepresentacao import ModelosEstatisticos, SentenceEmbeddings, WordEmbeddings
from ExtracaoConhecimento import Agrupamento, CompreensaoSemantica, CompreensaoEmocional

warnings.filterwarnings('ignore')
tqdm.pandas()

## Textos de mídias sociais
<br></br>
To Do

## Twitter

<br></br>
To Do

## Coletando Tweets

- Utilizaremos a Twitter API v2 para coletar os tweets 
- Acesse a [página de desenvolverdores](https://developer.twitter.com/) e obtenha suas credenciais de acesso
- Limitamos cada coleta a 10 tweets, mas todo o conteúdo dos tweets é adicionado (*appending*) ao arquivo local data/tweets.json
- Além dos campos *id* e *text* que estão presentes nos tweets por padrão, também solicitamos os campos *created_at, entities, geo, lang, public_metrics, source*. Para ver a lista completa de campos possíveis, acesse esta [página](https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet)
- Filtramos para selecionar apenas os tweets escritos em Inglês ( *lang = "en"* ) e que contenham o termo "nyc", que referencia a cidade de Nova Iorque
- Após coletar os tweets, extraímos os valores dos campos *text, timestamp_ms, ...* e retornamos no formato Pandas DataFrame. 

In [4]:
## Get Twitter API credentials

print("Informe seu 'API KEY'")
twitter_consumer_key = getpass.getpass()

print("Informe seu 'API KEY SECRET'")
twitter_consumer_secret = getpass.getpass()

print("Informe seu 'ACCESS TOKEN KEY'")
twitter_access_token_key = getpass.getpass()

print("Informe seu 'ACCESS TOKEN SECRET'")
twitter_access_token_secret = getpass.getpass()

print("Informe seu 'Bearer TOKEN'")
twitter_bearer_token = getpass.getpass()

Informe seu 'API KEY'
········
Informe seu 'API KEY SECRET'
········
Informe seu 'ACCESS TOKEN KEY'
········
Informe seu 'ACCESS TOKEN SECRET'
········
Informe seu 'Bearer TOKEN'
········


In [5]:
# collect tweets
df_tweets = ExtracaoDados().twitter(
    twitter_consumer_key, 
    twitter_consumer_secret, 
    twitter_access_token_key, 
    twitter_access_token_secret, 
    twitter_bearer_token
)

df_tweets.tail()

Unauthorized: 401 Unauthorized
Unauthorized

## Reddit

TODO

## Coletando Posts de \subreddits

- Utilizaremos a [API do Reddit](https://www.reddit.com/dev/api) para coletar posts
- Você deverá criar uma conta para acessar a API em [reddit.com](https://reddit.com)
- Depois de criar a conta, obtenha os Client ID e o Client Secret
- No exemplo a seguir, coletamos os top 100 posts de 5 subreddits, contendo o texto, url, número de comentários, data de criação e score (número de upvotes do post)
- Os dados são salvos no arquivo local data/reddit_posts.csv

In [13]:
## Get Reddit API credentials

print("Informe seu 'CLIENT ID'")
REDDIT_CLIENT_ID = getpass.getpass()

print("Informe seu 'CLIENT SECRET'")
REDDIT_CLIENT_SECRET = getpass.getpass()

Informe seu 'CLIENT ID'
········
Informe seu 'CLIENT SECRET'
········


In [17]:
# Subreddits with serious/relevant discussions
subreddits = [
    'politics',
    'AskHistorians',
    'changemyview',
    'COVID19',
    'EverythingScience',
    'science'
]

# collect top 100 posts from each subreddit
df_reddit_posts = ExtracaoDados().reddit(
    REDDIT_CLIENT_ID, 
    REDDIT_CLIENT_SECRET,
    subreddits=subreddits,
    top_n=100
)

100%|███████████████████████████████████████████████████████| 6/6 [01:10<00:00, 11.68s/it]


In [18]:
# show some posts with text
df_reddit_posts[df_reddit_posts['length'] > 0].tail(5)

,created,url,title,score,num_comments,text,length
315,1.583273e+09,https://www.reddit.com/r/COVID19/comments/fd29...,Please consider downloading BOINC or folding@h...,3124,1203,Hello all.\n\nI believe this has been posted b...,3181
500,1.521000e+09,https://www.reddit.com/r/science/comments/84ai...,Physicist Stephen Hawking dies aged 76,199300,3851,We regret to hear that [Stephen Hawking died t...,998
503,1.511109e+09,https://www.reddit.com/r/science/comments/7e1j...,Raising the taxes of graduate students by as m...,124379,10935,Science and technology development has been th...,2592
557,1.602797e+09,https://www.reddit.com/r/science/comments/jbwl...,[Megathread] World's most prestigious scientif...,80112,4480,We have received numerous submissions concerni...,2288
562,1.437997e+09,https://www.reddit.com/r/science/comments/3ere...,"Science Ama Series: I am Stephen Hawking, theo...",79242,8604,I signed an open letter earlier this year impl...,2211


### Pré - Processamento

A tokenização é um método para...

In [ ]:
pipeline = PreProcessamento(dados)

tokens = pipeline.tokenizacao()


A stemmização é um método para...

In [ ]:
stem =  pipeline.stemmizacao()

### Modelos de Representação

Modelos Estatísticos

In [ ]:
ModelosEstatisticos()

Word Embeddings

In [ ]:
WordEmbeddings()

Sentence Embeddings

In [ ]:
SentenceEmbeddings()

### Extração de Conhecimento

### Aplicações